In [2]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')
#!pip install -U -r /projects/icesat2_boreal/dps/requirements_main.txt

# Launch DPS for mapBoreal.R

In [31]:
import os
import geopandas
import pandas as pd
import json
import datetime
!pip install xmltodict
import xmltodict

def local_to_s3(url, user='lduncanson'):
    ''' A Function to convert local paths to s3 urls'''
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')
def local_to_https(url, user='lduncanson'):
    ''' A Function to convert local paths to https urls'''
    return url.replace('/projects/my-private-bucket', f'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/{user}')
def local_to_https_uswest2(url, user='lduncanson'):
    ''' A Function to convert local paths to https us-west-s urls'''
    return url.replace('/projects/my-private-bucket', f'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/{user}')

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [32]:
#atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/lduncanson/dps_output/run_tile_atl08_ubuntu/master/2022/03/ATL08_filt_tindex_master.csv')

#atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv')
#my-public-bucket/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv
atl08_filt_tindex_master =   pd.read_csv('/projects/shared-buckets/lduncanson/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv')
topo_tindex_master =         pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv')
landsat_tindex_master =      pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/HLS/fall2022/HLS_stack_2022_v2/HLS_tindex_master.csv')
lc_tindex_master = pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/09/LC_tindex_master.csv')


# Convert al local_paths to s3
#.. for data produced by 'lduncanson' workspace
atl08_filt_tindex_master['https'] = [local_to_https_uswest2(local_path, user='lduncanson') for local_path in atl08_filt_tindex_master['local_path']]

#.. for data produced by 'nathanmthomas' workspace
for tindex_master in [topo_tindex_master, landsat_tindex_master, lc_tindex_master]:
    tindex_master['https'] = [local_to_https_uswest2(local_path, user='nathanmthomas') for local_path in tindex_master['local_path']]

# Use the ATL08 filtered tindex master list to tell you which tiles you'll run

In [33]:

NORWAY_TILE_LIST = pd.read_csv('/projects/my-public-bucket/misc_files/norway_tiles.csv').layer.tolist()

#NORWAY_TILE_LIST = [42,199,200,223,224,386,387,11,49,111,112,131]

DELTA_TILE_LIST = [3459]

BONA_TILE_LIST  = [3364,3365]

HEALY_TILE_LIST = [3552]

INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST + DELTA_TILE_LIST + BONA_TILE_LIST + HEALY_TILE_LIST
#INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST
#DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST
#len(DPS_INPUT_TILE_NUM_LIST)

#ALASKA_TILE_LIST =  list(range(3268,3272+1))+\
#                    list(range(3361,3366+1))+\
#                    list(range(3454,3459+1))+\
#                    list(range(3549,3555+1))+\
#                    list(range(3643,3648+1))

INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST + DELTA_TILE_LIST + BONA_TILE_LIST + HEALY_TILE_LIST
print(len(INPUT_EXPERIMENT_TILE_NUM_LIST))
DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST

FULL_TILE_LIST = atl08_filt_tindex_master['tile_num']

DPS_INPUT_TILE_NUM_LIST = [639, 640, 726, 727, 373, 405]

DPS_INPUT_TILE_NUM_LIST = FULL_TILE_LIST
DPS_INPUT_TILE_NUM_LIST = [3082,2918, 1854, 1789, 1262,34210, 33975,37,148,169,446,162,629,1761, 2651, 2970, 3604, 4080]
DPS_INPUT_TILE_NUM_LIST = [1050, 1105, 34210, 4245, 24959, 23828]
DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST

DPS_INPUT_TILE_NUM_LIST = [639, 640, 726, 727, 373, 405, 42]

DPS_INPUT_TILE_NUM_LIST = FULL_TILE_LIST



print(len(DPS_INPUT_TILE_NUM_LIST))



127
4881


In [34]:
# Check retrieval of s3 path with a tle_num
in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]

print(in_atl08_https)
print(in_topo_https)
print(in_landsat_https)
print(in_lc_https)

https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/dps_output/run_tile_atl08_ubuntu/tile_atl08/2022/10/28/15/35/23/844340/atl08_005_30m_filt_topo_landsat_20221028_37077.csv
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/master/2022/03/21/14/34/11/573782/Copernicus_37077_covars_cog_topo_stack.tif
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/HLS_stack_2022_v2/2022/10/18/17/39/09/607785/HLS_37077_06-01_09-15_2019_2021.tif
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/run_build_stack_ubuntu/master/2022/09/15/17/20/27/167470/esa_worldcover_v100_2020_37077_cog.tif


## Get files for boreal biomass models & boreal wide sample


In [35]:
bio_models_https = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models.tar'

train_data_https = 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/boreal_train_data_v8.csv'

In [36]:
bio_models_https

'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models.tar'

## Test from Terminal

In [60]:
# put latest test data in a test folder (outside repo)
# Terminal:
                           
# file 1: /projects/testing/input/atl08_004_30m_filt_topo_landsat_20220208_0042.csv
# file 2: /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif
# file 3: /projects/shared-buckets/nathanmthomas/alg_34_testing/Landsat8_42_comp_cog_2015-2020_dps.tif
#file 4: /projects/my-private-bucket/boreal_train_data_v3.csv
#file 5: /projects/my-private-bucket/bio_models.tar
#file 6: 42
# file 7:

# ./run_boreal_biomass.sh file1 file2 file3 file4 file5 file6 file7
#ATL08_CSV= /projects/testing/input/atl08_005_30m_filt_topo_landsat_20220322_0042.csv
#TOPO_TIF=/projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif
#LANDSAT_TIF= /projects/shared-buckets/nathanmthomas/alg_34_testing/Landsat8_42_comp_cog_2015-2020_dps.tif
#DO_SLOPE_VALID_MASK= 'TRUE'
#ATL08_SAMPLE_CSV=/projects/my-private-bucket/boreal_train_data_v3.csv
#in_tile_num=42
#in_tile_fn=/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg

#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 42 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
#echo $cmd
#eval $cmd

# Set the output merged CSV name to a var
#MERGED_ATL08_CSV=$(ls ${OUTPUTDIR}/atl08_005_30m_filt_merge_neighbors* | head -1)

# Run mapBoreal with merged CSV as input
#conda activate r-with-gdal


#Rscript /projects/icesat2_boreal/lib/mapBoreal.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 2 2 100 300 5 TRUE 50 3000 

#test swapping for terra
#Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 3 2 100 300 5 TRUE 100 3000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 

# of tiles to run: 115
# [3585, 4108, 1037, 28176, 4112, 31762, 1050, 2075, 28702, 31263, 29223, 29739, 30252, 3117, 30763, 3120, 2614, 2618, 2619, 1615, 1105, 1617, 2142, 612, 2148, 2149, 2151, 2152, 2153, 1146, 1662, 1159, 1672, 2698, 34443, 26280, 2217, 2223, 2224, 33975, 1721, 3770, 3771, 1214, 1731, 1733, 26827, 2779, 3301, 3302, 27368, 745, 3307, 1273, 2301, 27908, 1798, 1799, 1800, 3861, 28440, 3865, 31513, 28963, 31014, 29481, 2859, 29996, 30508, 1334, 4411, 3393, 3395, 2375, 3399, 1865, 1866, 1867, 2378, 2379, 2380, 2381, 3400, 3951, 3952, 34674, 3954, 2942, 895, 2944, 1933, 1934, 1937, 1938, 2457, 3486, 34210, 931, 1448, 26554, 4034, 4035, 1999, 981, 3029, 3030, 2009, 2010, 2011, 27098, 991, 992, 993, 2533, 27638]

In [61]:
# test of tile 1927
#ATL08_CSV= /projects/testing/input/atl08_005_30m_filt_topo_landsat_20220418_1927.csv
#TOPO_TIF=/projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_1927_covars_cog_topo_stack.tif 
#LANDSAT_TIF= /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_1927_06-15_09-01_2019_2021.tif
#DO_SLOPE_VALID_MASK= 'TRUE'
#ATL08_SAMPLE_CSV=/projects/my-private-bucket/boreal_train_data_v4.csv
#in_tile_num=42
#in_tile_fn=/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg

#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 1927 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn /projects/my-public-bucket/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
#echo $cmd
#eval $cmd

# Set the output merged CSV name to a var
#MERGED_ATL08_CSV=$(ls ${OUTPUTDIR}/atl08_005_30m_filt_merge_neighbors* | head -1)

# Run mapBoreal with merged CSV as input
#conda activate r-with-gdal


#Rscript /projects/icesat2_boreal/lib/mapBoreal.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_726_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v5.csv 2 2 100 300 5 TRUE 50 3000 

#test swapping for terra
#Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/shared-buckets/nathanmthomas/atl08_004_30m_filt_merge_neighbors_3885.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_3585_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_727_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v5.csv 2 2 100 300 5 TRUE 100 3000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 



In [62]:
#test on tile 727 and 726
#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 727 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
       
#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 726 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/    

## Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0726.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_726_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_726_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v7.csv 2 2 130 250 5 TRUE 50 5000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 

## Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0727.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_727_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_727_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v7.csv 2 2 130 250 5 TRUE 50 5000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 



## register via terminal
python /projects/register-algorithm.py /projects/icesat2_boreal/dps/alg_3-4/algorithm_config.yaml

## Commit with Tag for running
1) Add version name as a map_boreal_2022_v2 or whatever is appropriate - both to this notebook and algorithm config yaml

2) follow git instructions (every time!!):
git add changes
git commit -m 'message'
git tag -f map_boreal_2022_v3

git push
git push dps
git push origin -f map_boreal_2022_v3
git push dps -f map_boreal_2022_v3

3) if it looks weird check git log to make sure tag is at same place as origin and dps


## Run a DPS job


In [63]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_boreal_biomass_quick_v2'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-32gb'
    
    # Maybe we're using an input tile list that isnt built directly from an ATL08_filt CSV
    # so, check if we have filtered ATL08 for this tile:
    if INPUT_TILE_NUM in atl08_filt_tindex_master['tile_num'].to_list():
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
 
    else:
        print(f"No filtered ATL08 for tile {INPUT_TILE_NUM}. Moving on.." )
        continue
    
    # iters is the number of iterations of RH model fits; set to low (2-5) to test, 30-50 for production
    # 30 yields run times ~4 hours
        
    # ppside is the number of subtile splits. 2=4 subtiles, 3=9 subtiles
    # increasing ppside reduces memory, increases length to completion
    
    # minDOY is the min possible DOY to search for tile training data. The default is set to May 1
    # If sufficient data are found in growing season, minDOY will not be used
    
    # maxDOY is the max possible DOY to search for tile training data. The default is Sept 30.
    
    # max_sol_el is the max solar elevation possible to search. The default is 0
    
    # expand_training as TRUE sets iterative searching for training data up to min_n, starting with solar_el<0 and growing season and then:
    # searches solar_el up to max_sol_el then
    # searches iteratively one month later after growing season until maxDOY then
    # searches iteratively one month earlier before growing season until minDOY
    # if expand_training=FALSE the growing season sol_el<0 will be taken, and if none, no data will return
    
    # min_n is the tile training number desired, that expand_training will search until it is fulfilled (or the max available)
    
    #local_train_perc is the percent tile training data vs. boreal-wide training data. 100 = all local, 0 = all boreal wide
    
    
    in_param_dict = {
                                    'in_atl08_fn': 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv',
                                    #'in_atl08_fn': f"input/{os.path.basename(in_atl08_https)}",
                                    'in_topo_fn': f"input/{os.path.basename(in_topo_https)}",
                                    'in_landsat_fn': f"input/{os.path.basename(in_landsat_https)}",
                                    'in_lc_fn': f"input/{os.path.basename(in_lc_https)}",
                                    'in_atl08_fn_url': in_atl08_https,
                                    'in_topo_fn_url': in_topo_https,
                                    'in_landsat_fn_url': in_landsat_https,
                                    'in_lc_fn_url': in_lc_https,
                                    'DO_SLOPE_VALID_MASK': 'TRUE',
                                    'in_atl08_sample_fn':f"input/{os.path.basename(train_data_https)}",
                                    'in_atl08_sample_url':train_data_https,
                                    'in_tile_num': INPUT_TILE_NUM,
                                    'in_tile_fn_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                                    'in_tile_fn': 'input/boreal_tiles_v003.gpkg',
                                    'in_tile_field': 'tile_num',
                                    'iters': 31,
                                    'ppside': 2,
                                    'minDOY':130,
                                    'maxDOY':250,
                                    'max_sol_el':5,
                                    'expand_training': 'TRUE',
                                    'local_train_perc': 100,
                                    'min_n': 5000, 
                                    'boreal_vect': 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/shared/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'boreal_vect_fn': 'input/wwf_circumboreal_Dissolve_reprj.geojson'
                }
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='map_boreal_2022_v3',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )

    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 2, 25, 50, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        #local_path

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

# of input tiles for DPS: 4881
DPS run #: 1	| tile num: 37077	| submit status: success	| job id: f1c058b2-7fd1-4ee5-858b-07f4ff19252c
DPS run #: 2	| tile num: 3573	| submit status: success	| job id: 9c203a0b-466c-48ac-890e-8ea077e58f67
DPS run #: 25	| tile num: 3690	| submit status: success	| job id: d50577eb-0964-44fd-a332-9e0bc10a1770
DPS run #: 50	| tile num: 3819	| submit status: success	| job id: d6aed47a-5a83-41d1-9b91-efa69fe8ce0a
DPS run #: 100	| tile num: 1657	| submit status: success	| job id: 8dd0432b-2dda-4c20-a852-7737ccb8150e
DPS run #: 500	| tile num: 2398	| submit status: success	| job id: 9bf51bd3-5daf-4b72-bdb4-2a043cd5da1b
DPS run #: 1000	| tile num: 39464	| submit status: success	| job id: 5a13e40a-74b1-4c69-a832-1cd6e6ef9c2a
DPS run #: 1500	| tile num: 469	| submit status: success	| job id: 2a58e515-9210-4ef1-b1f7-403d9a97c6e1
DPS run #: 2000	| tile num: 3264	| submit status: success	| job id: 5c2c76d9-f33a-42b4-8634-317c812ce23c
DPS run #: 3000	| tile num: 1304	| 

,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,f1c058b2-7fd1-4ee5-858b-07f4ff19252c,1,37077,2022-11-11 20:28:07.656146,20,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,9c203a0b-466c-48ac-890e-8ea077e58f67,2,3573,2022-11-11 20:28:07.724709,20,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,1417c932-594e-4e27-b84d-6895e1d42e7c,3,26574,2022-11-11 20:28:07.806822,20,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,05d8621f-8d31-43d3-9354-267809c5f7c6,4,28187,2022-11-11 20:28:07.929625,20,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,94fa3c56-aeee-4b9c-9c43-c5ed3bb861fd,5,23548,2022-11-11 20:28:08.328174,20,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
...,...,...,...,...,...,...,...,...,...,...
0,success,200,fdca183b-a29b-440d-9b5c-ac168ade9edb,4877,3364,2022-11-11 21:04:33.864843,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,7c300dc8-c2df-4111-b13c-7099179f0855,4878,3856,2022-11-11 21:04:34.127113,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,427d4288-f54b-47ed-88aa-a88f963385ba,4879,3538,2022-11-11 21:04:34.350369,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,8f5be63f-adca-41b9-9c75-5857588f26d4,4880,3074,2022-11-11 21:04:34.554496,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb


## Assess DPS Results

In [145]:
%%time
#submit_results_df = pd.read_csv('/projects/my-public-bucket/DPS_run_boreal_biomass_v5_submission_results_4581_202205092316.csv')
submit_results_df = pd.concat(submit_results_df_list)

def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

Count total jobs:	4881
Count pending jobs:	0
Count running jobs:	0
Count succeeded jobs:	2387
Count failed jobs:	2494
% of failed jobs:	51.1
CPU times: user 54.5 s, sys: 3.43 s, total: 57.9 s
Wall time: 1min 53s


In [137]:
submit_results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4881 entries, 0 to 0
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   status            4881 non-null   object        
 1   http_status_code  4881 non-null   int64         
 2   job_id            4881 non-null   object        
 3   dps_num           4881 non-null   int64         
 4   tile_num          4881 non-null   int64         
 5   submit_time       4881 non-null   datetime64[ns]
 6   dbs_job_hour      4881 non-null   int64         
 7   algo_id           4881 non-null   object        
 8   user              4881 non-null   object        
 9   worker_type       4881 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 419.5+ KB


In [22]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Succeeded'].iloc[0].job_id).content)

OrderedDict([('wps:Result',
              OrderedDict([('@xmlns:ows', 'http://www.opengis.net/ows/2.0'),
                           ('@xmlns:schemaLocation',
                            'http://schemas.opengis.net/wps/2.0/wps.xsd'),
                           ('@xmlns:wps', 'http://www.opengis.net/wps/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('wps:JobID',
                            '5e4fd861-5d92-4e5d-b5b8-1d4e8e6dd463'),
                           ('wps:Output',
                            OrderedDict([('@id',
                                          'output-2022-05-06T21:13:23.330599'),
                                         ('wps:Data',
                                          ['http://maap-ops-workspace.s3-website-us-west-2.amazonaws.com/lduncanson/dps_output/run_boreal_biomass_v4_ubuntu/master/2022/05/06/21/13/23/330599',
                                           's3:

In [136]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[20].job_id).content)

{'wps:Result': {'@xmlns:ows': 'http://www.opengis.net/ows/2.0',
  '@xmlns:schemaLocation': 'http://schemas.opengis.net/wps/2.0/wps.xsd',
  '@xmlns:wps': 'http://www.opengis.net/wps/2.0',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'wps:JobID': '167d1e36-6d6e-4d4c-9549-b02dae4d038e',
  'wps:Output': {'@id': 'traceback',
   'wps:Data': 'Traceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/job_worker.py", line 1085, in run_job\n    monitoredRunner.join()\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/process.py", line 148, in join\n    res = self._popen.wait(timeout)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/popen_fork.py", line 57, in wait\n    return self.poll(os.WNOHANG if timeout == 0.0 else 0)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/popen_fork.py", line 33, in poll\n    pid, sts = os.waitpid(self.pid, flag)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/pool.py", l

## After successful DPS run, create AGB tindex

In [40]:
## create AGB tindex 
month_dir_str = '11'
alg_name_str = 'run_boreal_biomass_quick_v2_ubuntu'
maap_version_str = 'map_boreal_2022_v3'
#index_out_dir = os.path.join('/projects/my-private-bucket/dps_output/run_boreal_biomass_qiuck_v2_ubuntu/map_boreal_2022_v3/2022/', month_dir_str)
index_out_dir = '/projects/my-public-bucket/DPS_tile_lists/fall2022/'
!python /projects/icesat2_boreal/lib/build_tindex_master.py -t 'AGB' -alg_name $alg_name_str --maap_version $maap_version_str -y 2022 -m $month_dir_str -o $index_out_dir

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Building a list of tiles:
MAAP version:		map_boreal_2022_v3
Type:		AGB
Year:		2022
Month:		['11']
Days:		1-31

Output dir:  /projects/my-public-bucket/DPS_tile_lists/fall2022/
                                             s3_path  ...                                     file
0  s3://maap-ops-workspace/lduncanson/dps_output/...  ...   boreal_agb_202211111668203884_2348.tif
1  s3://maap-ops-workspace/lduncanson/dps_output/...  ...  boreal_agb_202211111668204477_37079.tif
2  s3://maap-ops-workspace/lduncanson/dps_output/...  ...   boreal_agb_202211111668204586_3943.tif
3  s3://maap-ops-workspace/lduncanson/dps_output/...  ...  boreal_agb_202211111668205557_26297.tif
4  s3://maap-ops-workspac

In [41]:
TEST_DPS  = True

if TEST_DPS:
    
    if True:
        #!python /projects/icesat2_boreal/lib/build_tindex_master_v2.py -t AGB -y 2022 -m $month_dir_str -o $index_out_dir
        t = pd.read_csv(os.path.join(index_out_dir,'AGB_tindex_master.csv'))
        COMPLETED_TILES = t.tile_num.to_list()
        NEED_TILES = list(set(DPS_INPUT_TILE_NUM_LIST) - set(COMPLETED_TILES))

        #print(NEED_TILES)
        DPS_INPUT_TILE_NUM_LIST = NEED_TILES
    
else:
    DPS_INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST
   

#DPS_INPUT_TILE_NUM_LIST = [248, 273, 272, 271, 324]
print(f"# of tiles to run: {len(DPS_INPUT_TILE_NUM_LIST)}\n", DPS_INPUT_TILE_NUM_LIST)


# of tiles to run: 817
 [4, 4100, 11, 4108, 13, 2062, 2063, 16, 17, 34833, 2064, 4118, 26, 27, 28699, 28, 28701, 28702, 31, 4129, 28703, 28707, 37, 38, 24615, 39, 40, 28709, 24619, 43, 45, 46, 47, 2094, 4138, 4137, 4142, 4149, 54, 36917, 4152, 2105, 2107, 4157, 65, 2113, 67, 2122, 4176, 82, 34899, 34900, 85, 2133, 87, 88, 34903, 2138, 2139, 41052, 94, 24670, 2145, 24673, 99, 4196, 24678, 106, 2155, 108, 4204, 4207, 4208, 4209, 4211, 4212, 117, 2165, 4214, 2168, 4215, 127, 128, 2195, 4244, 4245, 4250, 2206, 159, 4256, 2210, 2211, 2213, 4262, 2216, 4266, 4268, 2221, 177, 2227, 181, 183, 184, 4286, 192, 2240, 4288, 4293, 4295, 26824, 201, 26825, 26826, 4296, 205, 2254, 4299, 37071, 209, 26834, 211, 2257, 213, 2261, 4304, 2264, 2265, 218, 2267, 220, 26841, 225, 2274, 4323, 228, 4325, 2280, 2283, 240, 2292, 2293, 4342, 4343, 4344, 255, 4358, 4359, 265, 33033, 4361, 268, 271, 4372, 2326, 279, 37142, 37143, 2332, 2333, 4380, 287, 41248, 291, 293, 2341, 2343, 296, 2345, 4389, 28969, 4396, 4395

## Re-run Failed Tiles in DPS

In [38]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_boreal_biomass_quick_v2'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-32gb'
    
    # Maybe we're using an input tile list that isnt built directly from an ATL08_filt CSV
    # so, check if we have filtered ATL08 for this tile:
    if INPUT_TILE_NUM in atl08_filt_tindex_master['tile_num'].to_list():
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
 
    else:
        print(f"No filtered ATL08 for tile {INPUT_TILE_NUM}. Moving on.." )
        continue
    
    # iters is the number of iterations of RH model fits; set to low (2-5) to test, 30-50 for production
    # 30 yields run times ~4 hours
        
    # ppside is the number of subtile splits. 2=4 subtiles, 3=9 subtiles
    # increasing ppside reduces memory, increases length to completion
    
    # minDOY is the min possible DOY to search for tile training data. The default is set to May 1
    # If sufficient data are found in growing season, minDOY will not be used
    
    # maxDOY is the max possible DOY to search for tile training data. The default is Sept 30.
    
    # max_sol_el is the max solar elevation possible to search. The default is 0
    
    # expand_training as TRUE sets iterative searching for training data up to min_n, starting with solar_el<0 and growing season and then:
    # searches solar_el up to max_sol_el then
    # searches iteratively one month later after growing season until maxDOY then
    # searches iteratively one month earlier before growing season until minDOY
    # if expand_training=FALSE the growing season sol_el<0 will be taken, and if none, no data will return
    
    # min_n is the tile training number desired, that expand_training will search until it is fulfilled (or the max available)
    
    #local_train_perc is the percent tile training data vs. boreal-wide training data. 100 = all local, 0 = all boreal wide
    
    
    in_param_dict = {
                                    'in_atl08_fn': 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv',
                                    #'in_atl08_fn': f"input/{os.path.basename(in_atl08_https)}",
                                    'in_topo_fn': f"input/{os.path.basename(in_topo_https)}",
                                    'in_landsat_fn': f"input/{os.path.basename(in_landsat_https)}",
                                    'in_lc_fn': f"input/{os.path.basename(in_lc_https)}",
                                    'in_atl08_fn_url': in_atl08_https,
                                    'in_topo_fn_url': in_topo_https,
                                    'in_landsat_fn_url': in_landsat_https,
                                    'in_lc_fn_url': in_lc_https,
                                    'DO_SLOPE_VALID_MASK': 'TRUE',
                                    'in_atl08_sample_fn':f"input/{os.path.basename(train_data_https)}",
                                    'in_atl08_sample_url':train_data_https,
                                    'in_tile_num': INPUT_TILE_NUM,
                                    'in_tile_fn_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                                    'in_tile_fn': 'input/boreal_tiles_v003.gpkg',
                                    'in_tile_field': 'tile_num',
                                    'iters': 31,
                                    'ppside': 2,
                                    'minDOY':130,
                                    'maxDOY':250,
                                    'max_sol_el':5,
                                    'expand_training': 'TRUE',
                                    'local_train_perc': 100,
                                    'min_n': 5000, 
                                    'boreal_vect': 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/shared/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'boreal_vect_fn': 'input/wwf_circumboreal_Dissolve_reprj.geojson'
                }
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='map_boreal_2022_v3',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )

    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 2, 25, 50, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        #local_path

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

# of input tiles for DPS: 958
DPS run #: 1	| tile num: 4	| submit status: success	| job id: 48638479-f72f-40a5-bd1a-a2b3f1d493f1
DPS run #: 2	| tile num: 4100	| submit status: success	| job id: cd1cf5d5-6308-40bc-9b37-f7aecc349600
DPS run #: 25	| tile num: 37	| submit status: success	| job id: d25ad3eb-6fda-43d5-afef-ce37ba39ff2c
DPS run #: 50	| tile num: 2122	| submit status: success	| job id: dbffc866-050f-46e0-80dc-0d8495244f73
DPS run #: 100	| tile num: 167	| submit status: success	| job id: df34625b-c997-4c19-a5db-4fd7dfbddd22
DPS run #: 500	| tile num: 886	| submit status: success	| job id: d14d66a0-80fd-4c46-ace0-bbd761795bae
Current time:	202211150536


ImportError: cannot import name 'CompressionOptions' from 'pandas._typing' (/opt/conda/lib/python3.7/site-packages/pandas/_typing.py)

## Re-Assess Re-submission
Continue with last few steps until all jobs successful: create tindex, create new tile list, re-run missing tiles in DPS, re-assess

In [55]:
%%time
#submit_results_df = pd.read_csv('/projects/my-public-bucket/DPS_run_boreal_biomass_quick_v2_submission_results_2494_202211132259.csv')
submit_results_df = pd.concat(submit_results_df_list)

def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

Count total jobs:	958
Count pending jobs:	0
Count running jobs:	159
Count succeeded jobs:	285
Count failed jobs:	514
% of failed jobs:	64.33
CPU times: user 11.7 s, sys: 755 ms, total: 12.4 s
Wall time: 23.6 s


## Get other lists just of missing tiles (various ways)

In [22]:
need_tindex_master = pd.read_csv('/projects/my-public-bucket/DPS_tile_lists/Need_AGB_tindex_master.csv')
print(len(need_tindex_master))

INPUT_TILE_NUM_LIST = need_tindex_master.tile_num.tolist()

# Remove duplicate tile nums
INPUT_TILE_NUM_LIST = list(set(INPUT_TILE_NUM_LIST))

print(len(INPUT_TILE_NUM_LIST))

INPUT_TILE_NUM_LIST_NEED = INPUT_TILE_NUM_LIST

1
1


In [9]:
weird_tiles = [4304, 4305, 4221, 4220, 1785, 1718, 1720, 1661, 1257, 1318, 1317]
werid_tiles = [1255, 1196, 949, 1062, 1063, 1005, 950, 1004]
INPUT_TILE_NUM_LIST = weird_tiles
len(INPUT_TILE_NUM_LIST)

11

In [10]:
# Get all boreal tiles
ATL08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv')
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.
select_needs = [3360,2994,3190,2840,3012,3014,3017,2932,1261,1263,1264,988,978,794, 380,378,411,821,861,
                812,765,764,1308,1302,1469,1406,2495,2883,2965,3321,3509,3510,3327,3335,2976,2906,2907,2894,2814,4253,4293,4403,4440,4408,4372,4477,3986]

# Remove bad tiles
boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    
#print(boreal_tile_index.head())
tile_matches_select_needs = boreal_tile_index.merge(ATL08_filt_tindex_master[ATL08_filt_tindex_master['tile_num'].isin(select_needs)], how='right', on='tile_num')
print(len(tile_matches_select_needs))
select_needs_filt = print([t for t in tile_matches_select_needs.tile_num])

45
[978, 988, 978, 988, 1263, 1261, 1264, 794, 2932, 3190, 3012, 3014, 3360, 3017, 378, 765, 812, 821, 764, 861, 1302, 1308, 1469, 2894, 2883, 2965, 2976, 3327, 3321, 3510, 3509, 4372, 4440, 4477, 2994, 1406, 2840, 411, 380, 3335, 2495, 4403, 2906, 2907, 2814]


In [32]:
#select_needs_filt = [978, 988, 978, 988, 1263, 1261, 1264, 794, 2932, 3190, 3012, 3014, 3360, 3017, 378, 765, 812, 821, 764, 861, 1302, 1308, 1469, 2894, 2883, 2965, 2976, 3327, 3321, 3510, 3509, 4372, 4440, 4477, 2994, 1406, 2840, 411, 380, 3335, 2495, 4403, 2906, 2907, 2814]

print(len(select_needs_filt))

import numpy as np
tile_nums_missing_but_wont_run = np.setdiff1d(select_needs, select_needs_filt)
print(len(tile_nums_missing_but_wont_run))
print(tile_nums_missing_but_wont_run)

tile_nums_missing_periphery = np.setdiff1d(INPUT_TILE_NUM_LIST_NEED, select_needs_filt)
print(len(tile_nums_missing_periphery))
print(tile_nums_missing_periphery)
INPUT_TILE_NUM_LIST = tile_nums_missing_periphery
len(INPUT_TILE_NUM_LIST)    

45
4
[3986 4253 4293 4408]
70
[  65  106  127  250  361  417  493  566  568  618  741  753  786  798
  801  830  848  876  877  927 1273 1287 1318 1402 1438 1439 1440 1604
 1848 1850 1852 1936 2425 2426 2952 3124 3355 3540 3542 3634 3728 3823
 3845 3846 3895 3916 4004 4007 4018 4083 4176 4204 4250 4256 4307 4325
 4342 4395 4417 4444 4450 4451 4463 4467 4468 4483 4499 4510 4527 4534]


70

In [18]:
INPUT_TILE_NUM_LIST = [4440,4372,4477]

## Send output cog.tif tiles somewhere

In [12]:
import glob
out_files = glob.glob("/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/**/*cog.tif", recursive=True)

In [13]:
print(out_files)

['/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/02/23/49/35/272239/boreal_agb_20220202_0200_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/01/42/37/290043/boreal_agb_20220203_0224_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/01/52/46/430279/boreal_agb_20220203_0387_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/02/21/50/348844/boreal_agb_20220203_0386_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/01/42/237812/boreal_agb_20220203_0199_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/06/47/336472/boreal_agb_20220203_0223_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/08/52/15295

In [ ]:
TEST_SUBDIR = 'add_solar'
OUTDIR = f'/projects/my-private-bucket/dps_output/run_boreal_biomass_v2_ubuntu/master/2022/{TEST_SUBDIR}'
!python /projects/icesat2_boreal/lib/build_tindex_master.py -y 2022 -m $TEST_SUBDIR -o $OUTDIR